# 6 WorkFlow  Gerencial, futuro=JULIO

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [3]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [4]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


ln: failed to create symbolic link '/content/buckets/b1/labo1': File exists


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 29 02:48:38 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660358,35.3,1454494,77.7,1454494,77.7
Vcells,1226521,9.4,8388608,64.0,1975136,15.1


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

if ( !require("readxl")) install.packages("readxl")
require("readxl") # Lectura del dataset Indice-FACPCE.xlsx

if ( !require("dplyr")) intall.packages("dplyr")
require("dplyr") # Necesaria para usar la función mutate y el pipe %>% - Codigo
# de corrección monetria en la seccion 6.3.1.3



Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros

Si es gerente, no cambie nada

Si es Analista, cambie el nombre del dataset

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 163733

PARAM$experimento <- "Exp_Final_V1"
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

In [7]:
head(dataset)

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29186441,202005,1,61,282,2354.94,43761.59,1682.14,-139.02,712.60,⋯,0,20,47,0,6507,0.00,0,7671,1043.97,CONTINUA
29187961,202005,1,61,312,5066.31,47268.33,3875.62,-3536.03,4551.11,⋯,0,228,232,0,5939,2017.56,0,8090,7495.47,CONTINUA
29193101,202005,1,67,365,4809.42,37996.66,6982.73,-2962.24,1109.46,⋯,0,133,149,0,2026,2838.66,0,2026,1653.93,CONTINUA
29193281,202005,1,54,158,2979.52,88436.59,3350.01,-908.55,592.70,⋯,2,10,115,0,3125,844.56,0,3350,2029.29,CONTINUA
29198891,202005,1,54,312,3008.70,22567.32,273.68,-643.87,2975.52,⋯,0,124,32,0,7202,234.60,0,7242,1395.87,CONTINUA
29205441,202005,1,72,6,394.19,1607.93,159.49,-9.92,210.01,⋯,0,41,32,0,157,0.00,0,157,58.65,CONTINUA


In [8]:
# Ejecuta un pequeño script JavaScript desde R para mantener activa la sesión
IRdisplay::display_javascript

function (data = NULL, file = NULL) 
display_raw("application/javascript", FALSE, data, file)
<bytecode: 0x59d74a28ca08>
<environment: namespace:IRdisplay>

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA a los valores, es la "anti imputación de valores faltantes"
<br> Se decide continuar con el método por defecto, en virtud de la recomendación de Patricia que experimento el Problema  #01
“Catastrophe Analysis”

In [9]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

In [10]:
dataset[, lapply(.SD, class)]

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
integer,integer,integer,integer,integer,numeric,numeric,numeric,numeric,numeric,⋯,integer,integer,integer,integer,integer,numeric,integer,integer,numeric,character


In [11]:
str(dataset)

Classes ‘data.table’ and 'data.frame':	273666 obs. of  32 variables:
 $ numero_de_cliente          : int  29186441 29187961 29193101 29193281 29198891 29205441 29207901 29243631 29251231 29257551 ...
 $ foto_mes                   : int  202005 202005 202005 202005 202005 202005 202005 202005 202005 202005 ...
 $ internet                   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ cliente_edad               : int  61 61 67 54 54 72 82 54 82 44 ...
 $ cliente_antiguedad         : int  282 312 365 158 312 6 312 110 312 187 ...
 $ mrentabilidad              : num  2355 5066 4809 2980 3009 ...
 $ mrentabilidad_annual       : num  43762 47268 37997 88437 22567 ...
 $ mcomisiones                : num  1682 3876 6983 3350 274 ...
 $ mactivos_margen            : num  -139 -3536 -2962 -909 -644 ...
 $ mpasivos_margen            : num  713 4551 1109 593 2976 ...
 $ cproductos                 : int  7 7 9 9 7 7 7 11 9 7 ...
 $ mcuenta_corriente          : num  0 0 0 8140 0 ...
 $ mcaja_ahorro             

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [16]:
# Descargar el archivo Indice-FACPCE.xlsx desde el repositorio

# Crear la carpeta data si no existe
dir.create("./data", showWarnings = FALSE, recursive = TRUE)

# URL del archivo en tu repositorio GitHub
url_facpce <- "https://github.com/Roxyalberti/labo2025v/raw/main/Proyecto-ML-Churn-Optimacion/data/Indice-FACPCE.xlsx"

# Descargar el archivo
download.file(
  url = url_facpce,
  destfile = "./data/Indice-FACPCE.xlsx",
  mode = "wb"  # modo binario para archivos Excel
)

# Verificar que se descargó
if(file.exists("./data/Indice-FACPCE.xlsx")) {
  print("Archivo Indice-FACPCE.xlsx descargado correctamente")
} else {
  stop("Error al descargar el archivo Indice-FACPCE.xlsx")
}

[1] "Archivo Indice-FACPCE.xlsx descargado correctamente"


In [19]:
# Implementación de Corrección Monetaria (Índice FACPCE)

# 1. Definición de las variables monetarias a corregir (candidatas)
vars_ajustar_candidatas <- c(
  "Visa_mconsumo", "Visa_mfinanciacion", "Visa_madelantok", "Visa_mpagominimo",
  "Master_mpagominimo", "mcomisiones", "mcuenta_corriente",
  "mtarjeta_master_consumo", "mprestamos_personales", "mcuentas_saldo",
  "mcaja_ahorro", "mcaja_usd", "mcomisiones_proximas",
  "mtarjeta_visa_descuentos", "mrentabilidad_annual", "mrentabilidad",
  "mactivos_margen", "mpasivos_margen", "mcuentas_margen", "mpayroll", "mpayroll2",
  "mtarjeta_visa_consumo"
)

# FILTRAR solo las variables que realmente existen en el dataset
vars_ajustar <- intersect(vars_ajustar_candidatas, colnames(dataset))

print(paste("Variables a ajustar:", length(vars_ajustar)))
print(vars_ajustar)


# 2. Cargar la tabla de factores FACPCE y Limpieza
tb_correccion <- readxl::read_excel(
  path = "./data/Indice-FACPCE.xlsx",
  sheet = 1
) %>%
  dplyr::mutate(
    coeficiente_de_ajuste = gsub(",", ".", coeficiente_de_ajuste),
    coeficiente_de_ajuste = gsub("[^0-9.]", "", coeficiente_de_ajuste),
    coeficiente_de_ajuste = as.numeric(coeficiente_de_ajuste)
  ) %>%
  dplyr::mutate(
    foto_mes = as.character(foto_mes),
    foto_mes = gsub("[^0-9]", "", foto_mes),
    foto_mes = substr(foto_mes, 1, 6)
  ) %>%
  as.data.table()


# 3. Limpieza de foto_mes en el dataset principal para asegurar el merge
dataset[, foto_mes := as.character(foto_mes)]
dataset[, foto_mes := gsub("[^0-9]", "", foto_mes)]
dataset[, foto_mes := substr(foto_mes, 1, 6)]


# 4. Merge por foto_mes
dataset <- merge(
  dataset,
  tb_correccion[, .(foto_mes, coeficiente_de_ajuste)],
  by = "foto_mes",
  all.x = TRUE
)


# 5. Aplicar la corrección por MULTIPLICACION (solo a las variables que existen)
if(length(vars_ajustar) > 0) {
  # Convertir a numeric
  dataset[, (vars_ajustar) := lapply(.SD, as.numeric), .SDcols = vars_ajustar]

  # SOLUCIÓN: Aplicar la corrección usando .BY o guardando el coeficiente
  # Opción 1 (más clara): iterar por cada variable
  for(var in vars_ajustar) {
    dataset[, (var) := get(var) * coeficiente_de_ajuste]
  }

  print("Corrección monetaria aplicada correctamente")
} else {
  warning("No se encontraron variables monetarias para ajustar")
}

# 6. Eliminar la columna auxiliar
dataset[, coeficiente_de_ajuste := NULL]

[1] "Variables a ajustar: 14"
 [1] "Visa_mpagominimo"        "Master_mpagominimo"     
 [3] "mcomisiones"             "mcuenta_corriente"      
 [5] "mtarjeta_master_consumo" "mprestamos_personales"  
 [7] "mcuentas_saldo"          "mcaja_ahorro"           
 [9] "mrentabilidad_annual"    "mrentabilidad"          
[11] "mactivos_margen"         "mpasivos_margen"        
[13] "mpayroll"                "mtarjeta_visa_consumo"  
[1] "Corrección monetaria aplicada correctamente"


In [21]:
# Ver algunas estadísticas de las variables corregidas
summary(dataset[, .(mrentabilidad, mcomisiones, mactivos_margen)])

# Ver cuántos valores NA hay en las variables ajustadas
sapply(dataset[, vars_ajustar, with = FALSE], function(x) sum(is.na(x)))

 mrentabilidad        mcomisiones       mactivos_margen     
 Min.   :-205840.4   Min.   :-49922.8   Min.   :-166603.13  
 1st Qu.:    105.6   1st Qu.:   367.7   1st Qu.:  -2000.03  
 Median :   1597.4   Median :  1054.4   Median :   -553.77  
 Mean   :   2877.3   Mean   :  1753.0   Mean   :  -1095.05  
 3rd Qu.:   3749.4   3rd Qu.:  2365.7   3rd Qu.:      1.95  
 Max.   : 795085.9   Max.   :253106.9   Max.   : 422600.45  
 NA's   :15323       NA's   :15323      NA's   :15323       

Visa_mpagominimo      Master_mpagominimo             mcomisiones 
                  12391                   27876                   15323 
      mcuenta_corriente mtarjeta_master_consumo   mprestamos_personales 
                      0                   15323                       0 
         mcuentas_saldo            mcaja_ahorro    mrentabilidad_annual 
                  15323                       0                   15323 
          mrentabilidad         mactivos_margen         mpasivos_margen 
                  15323                   15323                   15323 
               mpayroll   mtarjeta_visa_consumo 
                      0                   15323

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [ ]:
# Definición de la función auxiliar de atributos presentes que existe debido a
# las modalidades que posee el dataset.
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}


# El mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  # Asume que foto_mes ya es un valor numérico (lo fue en la celda anterior)
  dataset[, kmes := foto_mes %% 100]


# Variable extraida de una tesis de maestria de Irlanda: mpayroll / cliente_edad
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

In [ ]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [ ]:
# Feature Engineering Historico (Lags y Deltas)

# Variables a las que se les calculará el Lag y Delta
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  # Excluir la primary key y la clase
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# 1. Lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by = numero_de_cliente,
  .SDcols = cols_lagueables
]

# 2. Lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by = numero_de_cliente,
  .SDcols = cols_lagueables
]

# 3. Deltas (valor actual - lag)
for (vcol in cols_lagueables)
{
  # Delta 1: valor actual - lag1
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]

  # Delta 2: valor actual - lag2
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202107  obviamente completo

* final_train =  [ 202005, 202105 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202105   completo, sin undersampling
   * training = [ 202005, 202104 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202105)

PARAM$trainingstrategy$training <- c(
  202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c(
  202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202107)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Curva de Ganancia

Genero las salidas

In [ ]:
# asigno clase ternaria
tb_prediccion[, clase_ternaria := dfuture$clase_ternaria ]

# asigno ganancias
tb_prediccion[, ganancia := -3000.0 ]
tb_prediccion[clase_ternaria=="BAJA+2", ganancia := 117000.0 ]

# reordeno, y acumulada
setorder( tb_prediccion, -prob )
tb_prediccion[, gan_acum := cumsum(ganancia)]

# media movil de ancho 400
tb_prediccion[,
  gan_suavizada := frollmean(
    x= gan_acum,
    n= 400,
    align= "center",
    na.rm= TRUE,
    hasNA= TRUE
  )
]


In [ ]:
tb_prediccion[, sum(!is.na(gan_suavizada) )]
tb_prediccion[, .N]

In [ ]:
# Este es la ganancia a reportar
resultado <- list()
resultado$ganancia_suavizada_max <- max( tb_prediccion$gan_suavizada, na.rm=TRUE )
options(digits= 8)
resultado$envios <- which.max( tb_prediccion$gan_suavizada)
resultado

In [ ]:
# grabo las ganancias
fwrite( tb_prediccion,
  file= "ganancias.txt",
  sep= "\t"
)

In [ ]:
# genero el grafico

tb_prediccion[, envios:= .I]

pdf("curva_de_ganancia.pdf")

plot(
  x= tb_prediccion$envios,
  y= tb_prediccion$gan_acum,
  type= "l",
  col= "gray",
  xlim= c(0, 6000),
  ylim= c(0, 8000000),
  main= paste0("Mejor gan prom= ", as.integer(resultado$ganancia_suavizada_max), " envios= ", resultado$envios),
  xlab= "Envios",
  ylab= "Ganancia",
  panel.first= grid()
)

dev.off()

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

PARAM$resultado <- resultado

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")